In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import transforms
import numpy as np
import os
import time

In [19]:
def load_Positions(k):
    df1=pd.read_csv(f'PositionsM{k:02}.dat', sep='\t')
    return df1


In [20]:
def Processing_Events(k): # рассчитываем массив, где для индекса i - i частицы значение - суммарная энергия фотопов
    df1 = pd.read_csv(f'PositionsM{k:02}.dat', sep='\t')
    print(len(df1))
    if(k != 24 or k != 25 or k !=  26): # объяснено тем, что файлы в M24,M25,M26 названы по-другому
        ans=np.zeros(len(df1)) # размер обусловлен начальным условием
        for i in range(len(df1)):  
            if (os.path.exists(f'EventM{k:02}{i:05}.dat') == False):
                continue
            else:
                df2 = pd.read_csv(f'EventM{k:02}{i:05}.dat', sep='\t')
                ans[i] = df2['Energy'].sum()
    else:
        for i in range(len(df1)):
                if (os.path.exists(f'M{k:02}Event{i:05}.dat') == False):
                     continue
                else:
                    df2 = pd.read_csv(f'M{k:02}Event{i:05}.dat',sep='\t')
                    ans[i] = df2['Energy'].sum()
    return ans 

In [21]:
def different_cells(x,y,size,delta,nbr):
    fig, axs = plt.subplots(nrows= size , ncols= size )
    k=1
    for i in range(size):
        for j in range(size):
            x_lim=np.random.rand()*min(x)
            y_lim=np.random.rand()*min(y)
            plt.subplot(size,size,k)
            k+=1
            plt.xlim([x_lim,x_lim+delta])
            plt.ylim([y_lim,y_lim+delta])
            plt.xticks([])
            plt.yticks([])
            plt.scatter(x,y)

    fig.savefig(f'saved_figure{delta}M{nbr}.png')

In [23]:
def comprasion_xy_to_xy_grid(x_grid,y_grid,E_grid,temp_x,temp_y,E,i): #переобозначение координат частиц в сетку
    for i in range(len(temp_x)):
        temp_x_1 = np.abs(x_grid - temp_x[i])
        temp_y_1 = np.abs(y_grid - temp_y[i])
        E_grid[temp_x_1.argmin(),temp_y_1.argmin()]+=E[i]
    return E_grid
    


In [24]:
def get_xy(k,i):
    df1 = pd.read_csv(f'PositionsM{k:02}.dat', sep='\t')
    return df1['Sc_x'][i] , df1['Sc_y'][i]

In [25]:
def creating_db_energy(k):
    df1 = pd.read_csv(f'PositionsM{k:02}.dat', sep='\t')
    temp_x=df1['Sc_x'].to_numpy()
    temp_y=df1['Sc_y'].to_numpy()
    temp_E=Processing_Events(k)
    x_grid = np.arange(-520,520,20)
    y_grid = np.arange(-520,520,20)
    E_grid = np.zeros([52,52])
    for i in range(len(temp_x)):
        temp_x_1 = np.abs(x_grid - temp_x[i])
        temp_y_1 = np.abs(y_grid - temp_y[i])
        E_grid[temp_x_1.argmin(),temp_y_1.argmin()]+=temp_E[i]
    return x_grid,y_grid,E_grid


In [26]:
def creating_map(k):
    plt.figure(figsize=(10,10))
    x,y,E=creating_db_energy(k)
    X, Y = np.meshgrid(x, y)
    E_temp = E.copy()
    E_temp[26:53,:] = np.flip(E[0:26,:]) # т.к. изначально только для правой половины (из условия)
    temp_temp = E_temp.max()
    for i in range(52):
        for j in range(52):
            if E_temp[i,j]>0.75*temp_temp:
                E_temp[i,j]=0.75*temp_temp
    #cрез по x=0 i=26
    temp_E_ratio=E_temp/(np.sum(E_temp)/2704)
    temp_sum=np.sum(E_temp[26,:])
    temp_E_x=E_temp[26,:]*52/temp_sum
    plt.plot(x,temp_E_x)
    plt.savefig(f'matrixM{k:02}_1.png')
    plt.clf()
    ptr=plt.contourf(Y,X, temp_E_ratio, cmap=plt.cm.jet, levels=200, extend='both')
    plt.colorbar(ptr)
    plt.savefig(f'matrixM{k:02}_2.png')
    plt.clf()
    #plt.subplot(2,2,4)
    temp_sum=np.sum(E_temp[:,26])
    temp_E_y=E_temp[:,26]*52/temp_sum
    plt.plot(x,temp_E_y)
    plt.savefig(f'matrixM{k:02}_3.png')
    plt.show()

In [ ]:
st=[7,9,10,11,24,25,26]
for a in st:
    creating_map(a)